# NII-CU Multispectral Aerial Person Detection Dataset

Follow this notebook to prepare NII-CU dataset.

**Description:**
- The National Institute of Informatics - Chiba University (NII-CU) Multispectral Aerial Person Detection Dataset consists of 5,880 pairs of aligned RGB+FIR (Far infrared) images captured from a drone flying at heights between 20 and 50 meters, with the cameras pointed at 45 degrees down. We applied lens distortion correction and a homography warping to align the thermal images with the RGB images. We then labeled the people visible on the images with rectangular bounding boxes. The footage shows a baseball field and surroundings in Chiba, Japan, recorded in January 2020.

**Annotations:**
- in .txt format (one file per one image pair):
```txt
x1	y1	x2	y2	type	occluded	bad
```
example:
```txt
4.27	111.52	145.07	371.38	2	1	0
136.53	367.65	435.2	841.48	2	0	0
```
|Field|Description|
|:----|:----|
|x1|left of box in pixels, referring to RGB image space|
|y1|top of box in pixels, referring to RGB image space|
|x2|right of box in pixels, referring to RGB image space|
|y2|bottom of box in pixels, referring to RGB image space|
|type|0 = person visible on both RGB and thermal; 1 = visible only on Thermal; 2 = visible only on RGB|
|occluded|0 = completely visible; 1 = partially occluded|
|bad|0 = good; 1 = bad, e.g. blurry and smeared due to motion blur|

**Table of content:**

0. Init - imports and data download
1. Data annotation cleaning
2. Data transformation
3. Data visualization


## 0. Init - imports and data download
Download sard.zip files and extract them to `data/source/NII-CU` dir. After extract data should look like this:
```
data
└───source
    └───NII-CU
        ├───4-channel
        │   ├───images
        │   │   ├───rgb
        │   │   │   ├───train
        │   │   │   └───val
        │   │   └───thermal
        │   └───labels
        │       ├───train
        │       └───val
        └───rgb-t
            │   ├───images
            │   ├───rgb
            │   │   ├───train
            │   │   └───val
            │   └───thermal
            └───labels
                ├───train
                └───val
```
Currently, the rgb-t directory is not used

In [1]:
# Uncomment below two lines to reload imported packages (in case of modifying them)
# %load_ext autoreload
# %autoreload 2

# Imports
import os
import random
import numpy as np
import pandas as pd
import shutil
import cv2
import pybboxes as pbx
from pathlib import Path

from prj_utils.consts import ROOT_DIR
from data_processing.image_processing import plot_xywhn_annotated_image_from_file, get_brightness_stats, copy_annotated_images, get_number_of_objects_stats

# Consts
TRAIN_RGB_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/images/rgb/train'
VAL_RGB_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/images/rgb/val'
TRAIN_THERMO_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/images/thermal/train'
VAL_THERMO_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/images/thermal/val'
TRAIN_LABELS_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/labels/train'
VAL_LABELS_DIR = f'{ROOT_DIR}/data/source/NII-CU/4-channel/labels/val'

TRAIN_PROCESSED_DIR = f'{ROOT_DIR}/data/processed/NII-CU/train'
VAL_PROCESSED_DIR = f'{ROOT_DIR}/data/processed/NII-CU/validate'
TEST_PROCESSED_DIR = f'{ROOT_DIR}/data/processed/NII-CU/test'

## 1. Data transformation
- Transform labels from voc .txt format to yolo .txt files
- Split train data into train and validate dataset

After this step processed data directory should look like this:
```
data
└───processed
    └───Sard
        ├───test
        │   ├───images
        │   └───labels
        ├───train
        │   ├───images
        │   └───labels
        └───validate
            ├───images
            └───labels
```

## 1.1 Transform labels to yolo .txt files

Yolo format:
- One *.txt file per image (if no objects in image, no *.txt file is required).
- One row per object.
- Each row is `class x_center y_center scaled_width scaled_height` format, separated by space.
- Box coordinates must be in normalized from 0 to 1. If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
- Bounding box in annotation in xywhn format.
- Class numbers are zero-indexed (start from 0).
- Files are saved into `data/NII-CU/processed/train` and `data/Sard/processed/test` to RGB or Thermal directories and images or labels subdirectory. Currently both RGB and Thermal annotations are the same.


In [2]:
def read_bboxes(path) -> list[list[float]]:
    with open(path, 'r') as file:
        labels = []
        for line in file:
            labels.append((float(element) for element in line.split('\t')[:4]))
        return labels

def convert_bboxes(bboxes, image_size):
    yolo_labels = []
    for bbox in bboxes:
        yolo_bbox = pbx.convert_bbox(bbox, image_size=image_size, from_type="voc", to_type="yolo")
        yolo_label = (0,) + yolo_bbox
        yolo_labels.append(yolo_label)
    return yolo_labels

def save_labels(labels, output_file):
    with open(output_file, 'w') as f:
        for label in labels:
            line = ' '.join([str(l) for l in label])
            f.write(f'{line}\n')

In [3]:
def process_directory(input_directory_rgb, input_directory_thermal, input_directory_labels, output_directory):
    Path(f'{output_directory}/RGB/images').mkdir(parents=True, exist_ok=True)
    Path(f'{output_directory}/RGB/annotations').mkdir(parents=True, exist_ok=True)
    Path(f'{output_directory}/Thermal/images').mkdir(parents=True, exist_ok=True)
    Path(f'{output_directory}/Thermal/annotations').mkdir(parents=True, exist_ok=True)

    files = [f for f in os.listdir(input_directory_labels) if os.path.isfile(os.path.join(input_directory_labels, f))]

    for labels_file in files:
        labels_filename = Path(labels_file).stem
        labels_filepath = os.path.join(input_directory_labels, labels_file)

        image_file = f'{labels_filename}.jpg'
        rgb_image_filepath = os.path.join(input_directory_rgb, image_file)
        thermal_image_filepath = os.path.join(input_directory_thermal, image_file)

        image = cv2.imread(rgb_image_filepath)
        height, width, channels = image.shape
        image_size = (width, height)

        output_rgb_image_filepath = f'{output_directory}/RGB/images/{image_file}'
        output_rgb_labels_filepath = f'{output_directory}/RGB/annotations/{labels_file}'
        output_thermal_image_filepath = f'{output_directory}/Thermal/images/{image_file}'
        output_thermal_labels_filepath = f'{output_directory}/Thermal/annotations/{labels_file}'

        bboxes = read_bboxes(labels_filepath)
        yolo_lables = convert_bboxes(bboxes, image_size)
        save_labels(yolo_lables, output_rgb_labels_filepath)
        save_labels(yolo_lables, output_thermal_labels_filepath)

        shutil.copyfile(rgb_image_filepath, output_rgb_image_filepath)
        shutil.copyfile(thermal_image_filepath, output_thermal_image_filepath)

        # plot_xywhn_annotated_image_from_file(output_rgb_image_filepath, output_rgb_labels_filepath)
        # plot_xywhn_annotated_image_from_file(output_thermal_image_filepath, output_thermal_labels_filepath)

process_directory(TRAIN_RGB_DIR, TRAIN_THERMO_DIR, TRAIN_LABELS_DIR, TRAIN_PROCESSED_DIR)
process_directory(VAL_RGB_DIR, VAL_THERMO_DIR, VAL_LABELS_DIR, VAL_PROCESSED_DIR)

## 1.2 Change validate set to test set and split train data into train and validate dataset

Rename directory `data/NII-CU/processed/validate` to `data/NII-CU/processed/test`
Move random probes from `data/NII-CU/processed/train` to `data/NII-CU/processed/validate`.

In [4]:
random.seed(1)
np.random.seed(1)

shutil.move(VAL_PROCESSED_DIR, TEST_PROCESSED_DIR)

filenames = [f for f in os.listdir(f'{TRAIN_PROCESSED_DIR}/RGB/annotations') if os.path.isfile(os.path.join(f'{TRAIN_PROCESSED_DIR}/RGB/annotations', f))]

split = int(0.82 * len(filenames))

np.random.shuffle(filenames)
train_filenames = filenames[:split]
val_filenames = filenames[split:]

Path(f'{VAL_PROCESSED_DIR}/RGB/images').mkdir(parents=True, exist_ok=True)
Path(f'{VAL_PROCESSED_DIR}/RGB/annotations').mkdir(parents=True, exist_ok=True)
Path(f'{VAL_PROCESSED_DIR}/Thermal/images').mkdir(parents=True, exist_ok=True)
Path(f'{VAL_PROCESSED_DIR}/Thermal/annotations').mkdir(parents=True, exist_ok=True)

for file in val_filenames:
    filename = Path(file).stem

    rgb_image_filepath = f'{TRAIN_PROCESSED_DIR}/RGB/images/{filename}.jpg'
    rgb_label_filepath = f'{TRAIN_PROCESSED_DIR}/RGB/annotations/{filename}.txt'
    thermal_image_filepath = f'{TRAIN_PROCESSED_DIR}/Thermal/images/{filename}.jpg'
    thermal_label_filepath = f'{TRAIN_PROCESSED_DIR}/Thermal/annotations/{filename}.txt'

    output_rgb_image_filepath = f'{VAL_PROCESSED_DIR}/RGB/images/{filename}.jpg'
    output_rgb_label_filepath = f'{VAL_PROCESSED_DIR}/RGB/annotations/{filename}.txt'
    output_thermal_image_filepath = f'{VAL_PROCESSED_DIR}/Thermal/images/{filename}.jpg'
    output_thermal_label_filepath = f'{VAL_PROCESSED_DIR}/Thermal/annotations/{filename}.txt'

    shutil.move(rgb_image_filepath, output_rgb_image_filepath)
    shutil.move(rgb_label_filepath, output_rgb_label_filepath)
    shutil.move(thermal_image_filepath, output_thermal_image_filepath)
    shutil.move(thermal_label_filepath, output_thermal_label_filepath)
